In [ ]:
from dotenv import load_dotenv
import os
from langchain.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
load_dotenv()

In [ ]:
loader = CSVLoader(file_path="./e-commerce-data.csv")
documents = loader.load()

# **all-MiniLM-L6-v2** model

In [ ]:
# practice
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(documents, embedding_function)
query = "How many quantity of the wallet is left?"
docs = db.similarity_search(query)
print(docs[0].page_content)


In [ ]:
query = "I am thirsty so want to buy some drinks."
docs = db.similarity_search(query)
print(docs[0].page_content)

# LangChain Expression with Chroma DB CSV (RAG) - **OpenAIEmbeddings** model

In [ ]:
embeddings_ = OpenAIEmbeddings()
loader = CSVLoader(file_path="./e-commerce-data.csv")
documents = loader.load()
db = Chroma.from_documents(documents, embeddings_)

In [ ]:
query = "I am thirsty so want to buy some drinks."
docs = db.similarity_search(query)
print(docs[0].page_content)

In [ ]:
retriever = db.as_retriever()

template = """
You are the best online shopping mall seller.
Answer the question based only on the following context and basically recommend the product with unit price:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
print(chain.invoke("I am thirsty so want to buy some drinks."))
print(chain.invoke("I need a hanger to hang."))
print(chain.invoke("Soon my friend's birthday."))

# Shopping with ai assistant

1. The users ask the list of products they want
2. AI provides the product recommendations & ask the user for the specific product and quantity
3. The AI processes the user's choice and calculates the total price based on the unit price and quantity
4. The AI informs the user of the total cost

In [38]:
# from langchain import ChatOpenAI, CSVLoader, Chroma, OpenAIEmbeddings, StrOutputParser, ChatPromptTemplate, RunnablePassthrough

# Step 1: Setup
embeddings_ = OpenAIEmbeddings()
loader = CSVLoader(file_path="./e-commerce-data.csv")
documents = loader.load()
db = Chroma.from_documents(documents, embeddings_)
retriever = db.as_retriever()

# Step 2: Define the initial prompt template
initial_template = """
You are the best online shopping mall seller.
Answer the question based only on the following context and basically show all the product with unit price:
{context}

Question: {question}
"""

# Step 3: Define the follow-up prompt template
follow_up_template = """
Let me know the name of the product and quantity you want.
"""


# Step 4: Find the selected product
def calculate_total_price(product_name, quantity, product_list):
    while True:
        found = False
        for product in product_list:
            if product_name.lower() in product['name'].lower():
                unit_price = product['price']
                total_price = unit_price * quantity
                found = True
                return total_price
        
        if not found:
            print(f"Product '{product_name}' not found. Please enter the correct product name or type 'end' to quit.")
            product_name = input("Enter the product name again or 'end' to quit: ")
            if product_name.lower() == 'end':
                print("Exiting the search process.")
                return None
    
    
# Step 5: Setting up the chain with the templates and model
initial_prompt = ChatPromptTemplate.from_template(initial_template)
# follow_up_prompt = ChatPromptTemplate.from_template(follow_up_template)


In [41]:

model = ChatOpenAI()
# Creating the product list
product_list = []

# Function to execute the conversation chain
def run_conversation(user_question):
    # Get initial recommendations
    initial_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | initial_prompt
        | model
        | StrOutputParser()
    )

    initial_response = initial_chain.invoke(user_question)
    
    print(f"AI: {initial_response}", flush=True) # Print initial response and flush to ensure the message appears before the input prompt

    # Ask for product choice
    print("AI: Let me know the name of the product and quantity you want.", flush=True)  
    # Extract product and quantity from user input
    product_name = input("Choose the product- only input the name:")
    print(f"User: {product_name}")
    quantity = int(input("How many stock do you want? - Just input the number:"))
    print(f"User:{quantity}")

    # Product list
    docs = db.similarity_search(user_question)
    # Parsing function to extract product details
    def parse_product(doc):
        lines = doc.page_content.split('\n')
        product_info = {}
        for line in lines:
            key, value = line.split(': ')
            key = key.strip().lower()
            value = value.strip()
            if key == 'name':
                product_info['name'] = value
            elif key == 'unitprice':
                product_info['price'] = float(value)
            elif key == 'quantity':
                product_info['quantity'] = int(value)
        return product_info


    for doc in docs:
        product_info = parse_product(doc)
        if product_info not in product_list:
            product_list.append(product_info)
    
    

    # Calculate total price
    total_price = calculate_total_price(product_name.strip(), quantity, product_list)
    
    if total_price is not None:
        print(f"AI: The price of {product_name.strip()} is ${total_price}.")
    else:
        print("AI: Sorry, I couldn't find the product you specified.")


In [42]:
# Step 6: Running the conversation with the user question
user_input = input("Ask what you want :)")
print(f"User: {user_input}")
run_conversation(user_input)

User: I am hungry. I want to eat something.
AI: Sure! Here are some lunch boxes with cutlery that you can enjoy:

1. Name: LUNCH BOX WITH CUTLERY FAIRY CAKES
   Quantity: 1
   Unit Price: $0.0

Feel free to choose any of these lunch boxes to satisfy your hunger!
AI: Let me know the name of the product and quantity you want.
User: LUNCH BOX WITH CUTLERY FAIRY CAKES
User:1
AI: The price of LUNCH BOX WITH CUTLERY FAIRY CAKES is $0.0.


# Evaluation

In [ ]:
# Print product list to verify
for product in product_list:
    print(product)
print()


# Question list
- I am hungry. I want to eat something.
- I am thristy so want to buy some drinks.
- Is there any wallet?
- Soon my friend's birthday.